In [1]:
!pip install termcolor > /dev/null
!pip install langchain
!pip install openai
!pip install langchain_experimental
!pip install tiktoken
!pip install faiss-cpu

In [15]:
from langchain.chat_models import ChatOpenAI
from langchain_experimental.generative_agents import (
    GenerativeAgent,
    GenerativeAgentMemory,
)
import logging

logging.basicConfig(level=logging.ERROR)


llm = ChatOpenAI(max_tokens=1500,model="gpt-4o",temperature=0.2)

In [12]:
import math

import faiss
from langchain.embeddings import OpenAIEmbeddings
from langchain.docstore import InMemoryDocstore
from langchain.retrievers import TimeWeightedVectorStoreRetriever
from langchain.vectorstores import FAISS

USER_NAME="CTO"


def relevance_score_fn(score: float) -> float:
    """Return a similarity score on a scale [0, 1]."""
    # This will differ depending on a few things:
    # - the distance / similarity metric used by the VectorStore
    # - the scale of your embeddings (OpenAI's are unit norm. Many others are not!)
    # This function converts the euclidean norm of normalized embeddings
    # (0 is most similar, sqrt(2) most dissimilar)
    # to a similarity function (0 to 1)
    return 1.0 - score / math.sqrt(2)


def create_new_memory_retriever():
    """Create a new vector store retriever unique to the agent."""
    # Define your embedding model
    embeddings_model = OpenAIEmbeddings(model="text-embedding-3-small")
    # Initialize the vectorstore as empty
    embedding_size = 1536
    index = faiss.IndexFlatL2(embedding_size)
    vectorstore = FAISS(
        embeddings_model.embed_query,
        index,
        InMemoryDocstore({}),
        {},
        relevance_score_fn=relevance_score_fn,
    )
    return TimeWeightedVectorStoreRetriever(
        vectorstore=vectorstore, other_score_keys=["importance"], k=15
    )

def interview_agent(agent: GenerativeAgent, message: str) -> str:
    """Help the notebook user interact with the agent."""
    new_message = f"{USER_NAME} says {message}"
    return agent.generate_dialogue_response(new_message)[1]

In [4]:
arch_memory = GenerativeAgentMemory(
    llm=llm,
    memory_retriever=create_new_memory_retriever(),
    verbose=False,
    reflection_threshold=8,  # we will give this a relatively low number to show how reflection works
)

arch = GenerativeAgent(
    name="Sr. Architect",
    age=40,
    traits="likes Domain-Driven Design,understand very well Team Topologies, talkative, supportive, and patient",
    status="looking for help developers to understand Domain-Driven Design",
    memory_retriever=create_new_memory_retriever(),
    llm=llm,
    memory=arch_memory,
)

/Users/claudioed/development/ai/advanced-retrieval/.venv/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(
`embedding_function` is expected to be an Embeddings object, support for passing in a function will soon be removed.
`embedding_function` is expected to be an Embeddings object, support for passing in a function will soon be removed.


In [5]:
print(arch.get_summary())

Name: Sr. Architect (age: 40)
Innate traits: likes Domain-Driven Design,understand very well Team Topologies, talkative, supportive, and patient
Summary: A Sr. Architect demonstrates deep technical expertise, strong leadership skills, effective communication, strategic thinking, and a commitment to quality and innovation.


In [7]:
developer_memory = GenerativeAgentMemory(
    llm=llm,
    memory_retriever=create_new_memory_retriever(),
    verbose=False,
    reflection_threshold=5,
)

developer = GenerativeAgent(
    name="Sr. Developer",
    age=34,
    traits="curious, helpful",  # You can add more persistent traits here
    status="Started to understand Domain-Driven Design",  # When connected to a virtual world, we can have the characters update their status
    llm=llm,
    memory=developer_memory,
    verbose=False,
)
   

`embedding_function` is expected to be an Embeddings object, support for passing in a function will soon be removed.


In [8]:
from typing import List

def run_conversation(agents: List[GenerativeAgent], initial_observation: str) -> None:
    """Runs a conversation between agents."""
    _, observation = agents[1].generate_reaction(initial_observation)
    print(observation)
    turns = 0
    while True:
        break_dialogue = False
        for agent in agents:
            stay_in_dialogue, observation = agent.generate_dialogue_response(
                observation
            )
            print(observation)
            # observation = f"{agent.name} said {reaction}"
            if not stay_in_dialogue:
                break_dialogue = True
        if break_dialogue:
            break
        turns += 1

In [16]:
agents = [arch, developer]
run_conversation(
    agents,
    "Sr Architect: Hi, Developer. Thanks for agreeing to meet with me today. I have a bunch of questions and am not sure where to start. Maybe you could first share about your experience with Domain-Driven Design?",
)

Sr. Developer said "Sure, I'd be happy to share my experience with Domain-Driven Design. Where would you like to start?"
Sr. Architect said "Thank you for your willingness to share your experience. Let's dive into the practical aspects of identifying and defining bounded contexts. Could you walk me through a specific example from one of your past projects where you successfully identified and defined bounded contexts? It would be helpful to understand the practical steps you took and any challenges you faced during the process."
Sr. Developer said "Of course, I'd be happy to walk you through a specific example. Let's take the e-commerce platform project I mentioned earlier. Initially, we started by gathering all relevant stakeholders, including product owners, domain experts, and developers, for a series of workshops.
Sr. Architect said "Thank you for setting the stage. Could you describe the initial steps you took during these workshops to identify the bounded contexts? Specifically, 

/Users/claudioed/development/ai/advanced-retrieval/.venv/lib/python3.12/site-packages/langchain_experimental/generative_agents/memory.py:147: SyntaxWarning: invalid escape sequence '\M'
  + "\Memories: {memory_content}"


KeyboardInterrupt: 